# Problem
Recurrent neural networks can also be used as **generative models**.

This means that in addition to being used for predictive models (making predictions) 
they can learn the sequences of a problem and 
then generate entirely new plausible sequences for the problem domain.

In this lesson we are going to use the dataset: ``Alice’s Adventures in Wonderland``.

We are going to learn the dependencies between characters and 
the conditional probabilities of characters in sequences 
so that we can in turn generate wholly new and original sequences of characters.

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import tensorflow as tf
import sys
import os

### First step: Loading and marshalling of dataset

In [ ]:
### First step: Loading of dataset and marshaling of they# load ascii text and covert to lowercase
### we need to load the ASCII text for the book into memory and convert all of the characters to 
### lowercase to reduce the vocabulary that the network must learn.
filename = "wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

### We cannot model the characters directly, instead we must convert the characters to integers
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))


# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)


# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]    
    seq_out = raw_text[i + seq_length]
  
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print(dataX[0])
print(dataY[0])
print("Total Patterns: ", n_patterns)


# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
print(X.shape)
# normalize
X = X / float(n_vocab)

# one hot encode the output variable
y = np_utils.to_categorical(dataY)
print(y.shape)
print(y[0])

In [ ]:
for idx,lista in enumerate(dataX):
    if len(lista) !=100:
        print("lists with diff lenght ",idx, len(lista))


### Second step: Create the model 

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.LSTM(256))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(y.shape[1], activation='softmax'))
#filename = "Alice_in_Wonderland_models/weights-improvement-05-1.9609-bigger.hdf5"
#model.load_weights(filename)  # note to self: it is also possible to save the model (save_model/load_model), 
                               # and then there is no need to defined the model before calling the load_weights()
model.compile(loss='categorical_crossentropy', optimizer='adam')

### Third step: define a callback which will save the intermedie weights of model

In [ ]:
# dirName = "round_one"
# if not os.path.exists(dirName):
#     os.mkdir(dirName)
#     print("Directory " , dirName ,  " Created ")
# else:    
#     print("Directory " , dirName ,  " already exists")
filepath="round_one/weights-improvement-{epoch:02d}-{loss:.4f}-bigger2.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

### Fourth step: Training of model

In [ ]:
# fit the model
model.fit(X, y, epochs=2, batch_size=64, callbacks=callbacks_list)

In [ ]:
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
print("start: ", start)
pattern = dataX[start]
print("pattern:", pattern)
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

In [ ]:
# generate characters
for i in range(100):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    #print(prediction)
    index = np.argmax(prediction)
    #print(index)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

In [ ]:
filepath="round_two/weights-improvement-{epoch:02d}-{loss:.4f}-bigger2.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
# fit the model
model.fit(X, y, epochs=100, batch_size=64, callbacks=callbacks_list)

In [ ]:
from keras.models import load_model
model = load_model("/content/round_one/weights-improvement-60-1.1718-bigger2.hdf5")

In [ ]:
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
print("start: ", start)
pattern = dataX[start]
print("pattern:", len(pattern))
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

In [ ]:
# generate characters
for i in range(100):
    x = np.reshape(pattern, (1, len(pattern), 1))
    #print("x shape: ",x.shape)
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

In [ ]:
X_train_nn = np.array(dataX)
X_train_nn = X_train_nn/ float(n_vocab)
print(X_train_nn.shape)

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(256, input_shape=(100,), activation="relu"))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(256, activation= "relu"))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(y.shape[1], activation='softmax'))
#filename = "Alice_in_Wonderland_models/weights-improvement-05-1.9609-bigger.hdf5"
#model.load_weights(filename)  # note to self: it is also possible to save the model (save_model/load_model), 
                               # and then there is no need to defined the model before calling the load_weights()
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.fit(X_train_nn, y, epochs=10, batch_size=64)

In [ ]:
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
print("start: ", start)
pattern = dataX[start]
print("pattern:", len(pattern))
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

In [ ]:
# generate characters
for i in range(100):
    x = np.reshape(pattern, (1, len(pattern)))
    #print("x shape: ",x.shape)
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")